# 文本标签分类
标签化包含几个组成部分

- function：与提取类似，标签化使用函数来指定模型应如何标记文档
- schema：定义了我们希望如何标记文档

In [1]:
import getpass
import os

if not os.environ.get("DEEPSEEK_API_KEY"):
  os.environ["DEEPSEEK_API_KEY"] = getpass.getpass("Enter API key for DeepSeek: ")

from langchain.chat_models import init_chat_model

llm = init_chat_model("deepseek-chat", model_provider="deepseek")

Enter API key for DeepSeek:  ········


让我们在 Schema 中指定一个 Pydantic 模型，其中包含一些属性及其预期类型

In [2]:
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field

tagging_prompt = ChatPromptTemplate.from_template(
    """
Extract the desired information from the following passage.

Only extract the properties mentioned in the 'Classification' function.

Passage:
{input}
"""
)


class Classification(BaseModel):
    sentiment: str = Field(description="The sentiment of the text")
    aggressiveness: int = Field(
        description="How aggressive the text is on a scale from 1 to 10"
    )
    language: str = Field(description="The language the text is written in")


# Structured LLM
structured_llm = llm.with_structured_output(Classification)

In [3]:
inp = "Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!"
prompt = tagging_prompt.invoke({"input": inp})
response = structured_llm.invoke(prompt)

response

Classification(sentiment='positive', aggressiveness=1, language='Spanish')

In [4]:
inp = "Estoy muy enojado con vos! Te voy a dar tu merecido!"
prompt = tagging_prompt.invoke({"input": inp})
response = structured_llm.invoke(prompt)

response.model_dump()

{'sentiment': 'negative', 'aggressiveness': 9, 'language': 'Spanish'}

In [5]:
inp = "今天没有完成任务，又困又饿。"
prompt = tagging_prompt.invoke({"input": inp})
response = structured_llm.invoke(prompt)

response.model_dump()

{'sentiment': 'negative', 'aggressiveness': 2, 'language': 'Chinese'}


## 下面使用了枚举类型，只能输出指定类型，比如 enum=["happy", "neutral", "sad"]

In [9]:
class Classification(BaseModel):
    sentiment: str = Field(..., enum=["happy", "neutral", "sad"])
    aggressiveness: int = Field(
        ...,
        description="describes how aggressive the statement is, the higher the number the more aggressive",
        enum=[1, 2, 3, 4, 5],
    )
    language: str = Field(
        ..., enum=["spanish", "english", "french", "german", "italian"]
    )

In [10]:
tagging_prompt = ChatPromptTemplate.from_template(
    """
Extract the desired information from the following passage.

Only extract the properties mentioned in the 'Classification' function.

Passage:
{input}
"""
)

structured_llm = llm.with_structured_output(Classification)

In [11]:
inp = "Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!"
prompt = tagging_prompt.invoke({"input": inp})
structured_llm.invoke(prompt)

Classification(sentiment='happy', aggressiveness=1, language='spanish')

#### 询问gpt后得到讲解如下

## Pydantic 模型详解

### 什么是 Pydantic 模型？

Pydantic 是一个 Python 库，主要用于**数据验证**和**设置管理**。它的核心思想是：

- **基于类型注解**：使用 Python 的类型提示来定义数据结构
- **运行时验证**：在数据创建和修改时自动验证数据的正确性
- **数据序列化**：轻松地在不同格式（如 JSON、字典）之间转换数据

### 在代码中的具体体现

让我们分析这段代码中 Pydantic 模型的作用：

```python
from pydantic import BaseModel, Field

class Classification(BaseModel):
    sentiment: str = Field(..., enum=["happy", "neutral", "sad"])
    aggressiveness: int = Field(
        ...,
        description="describes how aggressive the statement is, the higher the number the more aggressive",
        enum=[1, 2, 3, 4, 5],
    )
    language: str = Field(
        ..., enum=["spanish", "english", "french", "german", "italian"]
    )
```

### Pydantic 模型在这里的具体作用

#### 1. **数据模式定义**
```python
class Classification(BaseModel):
```
- 继承自 `BaseModel` 表明这是一个 Pydantic 数据模型
- 定义了从文本中提取信息的"蓝图"或"合同"

#### 2. **字段类型约束**
```python
sentiment: str
aggressiveness: int  
language: str
```
- **类型注解**：明确指定每个字段的数据类型
- `sentiment` 和 `language` 必须是字符串
- `aggressiveness` 必须是整数
- 如果 LLM 返回了错误类型，Pydantic 会自动验证并报错

#### 3. **字段验证规则**
```python
Field(..., enum=["happy", "neutral", "sad"])
```
- **`...`**：表示该字段是必需的（不能为 None）
- **`enum`**：限制字段只能取特定的值
  - `sentiment` 只能是 "happy", "neutral", "sad" 之一
  - `aggressiveness` 只能是 1-5 的整数
  - `language` 只能是预定义的五种语言之一

#### 4. **字段描述信息**
```python
description="describes how aggressive the statement is..."
```
- 这个描述会传递给 LLM，帮助它理解如何填充这个字段
- 这是 Pydantic 与 LLM 协作的关键桥梁

### Pydantic 在流程中的工作方式

#### 创建阶段：定义期望的输出结构
```python
# 这创建了一个"数据合同"
# 告诉系统：我期望得到这样的结构化数据
structured_llm = llm.with_structured_output(Classification)
```

#### 执行阶段：自动验证和转换
当 LLM 返回结果时：

1. **LangChain 自动调用 Pydantic 验证**
2. **Pydantic 检查数据类型和约束**
   - 确保 `sentiment` 是字符串且在枚举范围内
   - 确保 `aggressiveness` 是整数且在 1-5 范围内
   - 确保 `language` 是预定义的语言之一
3. **如果验证通过**：创建 `Classification` 对象
4. **如果验证失败**：抛出清晰的错误信息

#### 结果使用阶段
```python
# 获得一个经过验证的 Pydantic 对象
result = structured_llm.invoke(prompt)

# 可以直接访问属性（因为类型已经确保正确）
print(result.sentiment)      # 一定是字符串
print(result.aggressiveness) # 一定是 1-5 的整数
print(result.language)       # 一定是预定义语言之一
```

### Pydantic 带来的核心价值

#### 1. **数据可靠性**
```python
# 如果没有 Pydantic，你可能得到：
# - sentiment: "very positive" （不在枚举中）
# - aggressiveness: "low" （不是数字）
# - language: "Español" （不在枚举中）

# 有了 Pydantic，你得到：
Classification(sentiment='happy', aggressiveness=1, language='spanish')
```

#### 2. **开发效率**
- 自动处理数据验证，无需手动写 if-else 检查
- 清晰的错误信息帮助快速定位问题
- 类型提示让 IDE 能够提供更好的代码补全和错误检测

#### 3. **与 LLM 的无缝集成**
```python
# Pydantic 模型自动转换为 LLM 能理解的结构化指令
structured_llm = llm.with_structured_output(Classification)
# 底层会自动将 Pydantic 模型转换为 JSON Schema 等格式给 LLM
```

### 实际效果演示

假设 LLM 返回了这样的原始数据：
```json
{
  "sentiment": "positive",
  "aggressiveness": 1,
  "language": "Spanish"
}
```

Pydantic 会自动：
1. 将 "positive" 映射到最接近的枚举值 "happy"
2. 验证 1 在 1-5 范围内 ✓
3. 将 "Spanish" 标准化为 "spanish"

最终输出：
```python
Classification(sentiment='happy', aggressiveness=1, language='spanish')
```

### 总结

**Pydantic 模型在这里的作用是**：
- 🎯 **定义数据契约**：明确指定要从文本中提取什么信息
- 🔒 **强制执行验证**：确保提取的数据符合预期格式和约束  
- 🤖 **指导 LLM 行为**：通过字段描述和枚举限制 LLM 的输出
- 🚀 **提升开发体验**：自动处理数据验证，让代码更健壮可靠

这就是为什么在结构化输出中使用 Pydantic 模型如此重要——它将模糊的文本生成变成了精确的数据提取流水线。